In [41]:
import argparse
import cv2
import os
import numpy as np
import random
import math
import sys
import time
import matplotlib.pyplot as plt 
import shutil

NAMES = [ 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
         'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
         'U', 'V', 'W', 'X', 'Y', 'Z']

cube_list = []
cube_names = []
background_list = []
background_names = []

NUM_TOTAL_VARIATIONS = 100000
SAMPLES_PER_IMAGE = 3000
SAMPLE = True

In [42]:


def load_images_from_folder(folder, images, names):
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
            images.append(img)
            names.append(filename)
            #plt.imshow( img)
            #plt.show()
            
    return images

def rotation(image, angleInDegrees):
    h, w = image.shape[:2]
    img_c = (w / 2, h / 2)

    rot = cv2.getRotationMatrix2D(img_c, 360-angleInDegrees, 1)

    rad = math.radians(angleInDegrees)
    sin = math.sin(rad)
    cos = math.cos(rad)
    b_w = int((h * abs(sin)) + (w * abs(cos)))
    b_h = int((h * abs(cos)) + (w * abs(sin)))

    rot[0, 2] += ((b_w / 2) - img_c[0])
    rot[1, 2] += ((b_h / 2) - img_c[1])

    outImg = cv2.warpAffine(image, rot, (b_w, b_h), flags=cv2.INTER_LINEAR)
    return outImg

#yolo annotation format, normal: [class, x_m, y_m, d_x, d_y], here: +angle 
def writeYOLOAnnotation(textfile, idx, bb, angle):
    name_index = NAMES.index(os.path.splitext(cube_names[idx])[0])          
    with open(textfile, 'a') as the_file:
        the_file.write(str(name_index))
        the_file.write(' ')
        the_file.write(str(bb[0]))
        the_file.write(' ')
        the_file.write(str(bb[1]))
        the_file.write(' ')
        the_file.write(str(bb[2]))
        the_file.write(' ')
        the_file.write(str(bb[3]))
        the_file.write(' ')
        the_file.write(str(angle))
        the_file.write('\n')

def writeYOLOAnnotation_without_rot(textfile, idx, bb):
    name_index = NAMES.index(os.path.splitext(cube_names[idx])[0])          
    with open(textfile, 'a') as the_file:
        the_file.write(str(name_index))
        the_file.write(' ')
        the_file.write(str(bb[0]))
        the_file.write(' ')
        the_file.write(str(bb[1]))
        the_file.write(' ')
        the_file.write(str(bb[2]))
        the_file.write(' ')
        the_file.write(str(bb[3]))
        the_file.write(' ')
        the_file.write('\n')

def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
	invGamma = 1.0 / gamma
	table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")
	# apply gamma correction using the lookup table
	return cv2.LUT(image, table)

def updt(total, progress):
    """
    Displays or updates a console progress bar.

    Original source: https://stackoverflow.com/a/15860757/1391441
    """
    barLength, status = 20, ""
    progress = float(progress) / float(total)
    if progress >= 1.:
        progress, status = 1, "\r\n"
    block = int(round(barLength * progress))
    text = "\r[{}] {:.0f}% {}".format(
        "#" * block + "-" * (barLength - block), round(progress * 100, 0),
        status)
    sys.stdout.write(text)
    sys.stdout.flush()

In [46]:
folder_path = "/Users/paulschulz/Cube_detection/Dataset_home/base/"
images_path = "/Users/paulschulz/Cube_detection/Dataset_home/base/images"
labels_path = "/Users/paulschulz/Cube_detection/Dataset_home/base/labels"

cube_path = "/Users/paulschulz/Cube_detection/Dataset_home/cubes"
background_path = '/Users/paulschulz/Cube_detection/Dataset_home/backgrounds'



if os.path.exists(images_path):
    shutil.rmtree(images_path)
os.mkdir(images_path)

if os.path.exists(labels_path):
    shutil.rmtree(labels_path)
os.mkdir(labels_path)

In [47]:
os.chdir('/Users/paulschulz/Cube_detection/Dataset_home/base')

In [48]:
bg_rect = [100, 255, 1700, 700]
folder="/Users/paulschulz/Cube_detection/Dataset_home/backgrounds"
file_list=os.listdir(folder)
file_list.remove(".DS_Store")
cubes=load_images_from_folder(cube_path, cube_list, cube_names)

num_of_images=len(file_list)
for i in range(NUM_TOTAL_VARIATIONS):

    background_file_index=random.randint(0, (num_of_images-1))
    background_img=cv2.imread(os.path.join(folder,file_list[background_file_index]))
    background_img=cv2.cvtColor(background_img, cv2.COLOR_BGR2BGRA)
    
    if background_img.shape is not (1920, 1080):
        background_img = cv2.resize(background_img, (1920, 1080), interpolation = cv2.INTER_AREA)
    morphed_img=background_img.copy()

    
    image_name = str(i).zfill(8)
    textfile_name = str(i).zfill(8) +".txt"
    imagefile_name = str(i).zfill(8) +".jpg"

    
    num_of_samples = random.randint(1, 15)
    for y in range(num_of_samples):
        idx = random.randrange(len(cubes))
        cube = cubes[idx]

        ### RADNOM SCALING ###
        scale_percent = random.randint(80,250)

        #mu, sigma = 1, 0.1
        #w = np.random.normal(mu, sigma, 1)
        #h = np.random.normal(mu, sigma, 1)

        width = int(cube.shape[1] * scale_percent  / 100)
        height = int(cube.shape[0] * scale_percent / 100)
        dim = (width, height)
        
        # resize image
        cube = cv2.resize(cube, dim, interpolation = cv2.INTER_AREA)

        ### RANDOM BLUR ###
        if random.randint(0,1) == 1:
            ksize = random.randint(1,10 )
            if(ksize % 2 != 1):
                ksize = ksize+1
            cube = cv2.GaussianBlur(cube,(ksize,ksize),0)
        
        ### RANDOM ROTATING ###
        rot_degree = random.randint(0,360)
        (h, w) = cube.shape[:2]
        (cX, cY) = (w // 2, h // 2)
        rotated = rotation(cube, rot_degree)
        rad = math.radians(rot_degree)
        sin = math.sin(rad)
        cos = math.cos(rad)

        b_w = int((h * abs(sin)) + (w * abs(cos)))
        b_h = int((h * abs(cos)) + (w * abs(sin)))

        rand_pos = [random.randint(bg_rect[0], bg_rect[0]+ bg_rect[2]),random.randint(bg_rect[1], bg_rect[1]+ bg_rect[3])]

        y1, y2 = int(rand_pos[1]-b_h/2),  int(rand_pos[1]+b_h/2)
        x1, x2 = int(rand_pos[0]-b_w/2), int(rand_pos[0]+b_w/2)
        y1_, y2_ = int(rand_pos[1]-h/2),  int(rand_pos[1]+h/2)
        x1_, x2_ = int(rand_pos[0]-w/2), int(rand_pos[0]+w/2)

        trans_indices = rotated[...,3] != 0 # Where not transparent
        alpha_s = rotated[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
        
        for c in range(0, 3):
            morphed_img[y1:y2, x1:x2, c] = (alpha_s * rotated[:, :, c] + 
                                            alpha_l * morphed_img[y1:y2, x1:x2, c])

        
        bb = [(x1+(x2-x1)/2)/background_img.shape[1], (y1+(y2-y1)/2)/background_img.shape[0], (x2_-x1_)/background_img.shape[1], (y2_-y1_)/background_img.shape[0]]
        
        textfile = os.getcwd() + "/labels/" + textfile_name

        writeYOLOAnnotation_without_rot(textfile, idx, bb)
    
     
    if(num_of_samples == 0):
        textfile = os.getcwd() + "/labels/" + textfile_name
        with open(textfile, 'a') as the_file:
            the_file.write(' ')
    
    file = os.getcwd() + "/images/" + imagefile_name
    morphed_img_res = adjust_gamma(morphed_img, random.uniform(0.5,1.5))
    morphed_img_res = cv2.resize(morphed_img_res, (854, 480))

    cv2.imwrite(file, morphed_img_res)
    updt(NUM_TOTAL_VARIATIONS, i)
    
print("\n")

[--------------------] 0% 

<>:14: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:14: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/03/l63p1j7d4kz0crxt306fv33w0000gn/T/ipykernel_52120/887422610.py:14: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if background_img.shape is not (1920, 1080):


[####################] 100% 



In [52]:
os.chdir('/Users/paulschulz/Cube_detection/Dataset_home')

In [53]:
content_path = '/Users/paulschulz/Cube_detection/Dataset_home/ds_train'
os.mkdir(content_path)


images_path = '/Users/paulschulz/Cube_detection/Dataset_home/ds_train/images'
os.mkdir(images_path)
    
labels_path = '/Users/paulschulz/Cube_detection/Dataset_home/ds_train/labels'
os.mkdir(labels_path)

training_images_path = '/Users/paulschulz/Cube_detection/Dataset_home/ds_train/images/train'
validation_images_path = '/Users/paulschulz/Cube_detection/Dataset_home/ds_train/images/val'
training_labels_path = '/Users/paulschulz/Cube_detection/Dataset_home/ds_train/labels/train'
validation_labels_path = '/Users/paulschulz/Cube_detection/Dataset_home/ds_train/labels/val'

os.mkdir(training_images_path)
os.mkdir(validation_images_path)
os.mkdir(training_labels_path)
os.mkdir(validation_labels_path)

In [54]:
import os, shutil, random

# preparing the folder structure

full_data_path = '/Users/paulschulz/Cube_detection/Dataset_home/base/images/'

full_data_path_labels='/Users/paulschulz/Cube_detection/Dataset_home/base/labels/'
extension_allowed = '.jpg'
split_percentage = 90

files = []
ext_len = len(extension_allowed)


for r, d, f in os.walk(full_data_path):
    for file in f:
        if file.endswith(extension_allowed):
            strip = file[0:len(file) - ext_len]      
            files.append(strip)

#random.shuffle(files)

size = len(files)                   

split = int(split_percentage * size / 100)

print("copying training data")
for i in range(split):
    strip = files[i]
                         
    image_file = strip + extension_allowed
    src_image = full_data_path + image_file
    #shutil.copy(src_image, training_images_path) 
    shutil.move(src_image, training_images_path) 
                         
    annotation_file = strip + '.txt'
    src_label = full_data_path_labels + annotation_file
    #shutil.copy(src_label, training_labels_path) 
    shutil.move(src_label, training_labels_path) 

print("copying validation data")
for i in range(split, size):
    strip = files[i]
                         
    image_file = strip + extension_allowed
    src_image = full_data_path + image_file
    #shutil.copy(src_image, validation_images_path) 
    shutil.move(src_image, validation_images_path) 
    
                         
    annotation_file = strip + '.txt'
    src_label = full_data_path_labels + annotation_file
    #shutil.copy(src_label, validation_labels_path) 
    shutil.move(src_label, validation_labels_path) 

print("finished")

copying training data
copying validation data
finished


In [57]:
print(len(os.listdir("/Users/paulschulz/Cube_detection/Dataset_home/ds_train/images/val")))
print(len(os.listdir("/Users/paulschulz/Cube_detection/Dataset_home/ds_train/labels/train")))

10000
90000
